In [57]:
# %%
import numpy as np
from numpy.fft import rfft2, irfft2, rfftfreq, fftfreq
import matplotlib.pyplot as plt
import vtk
from noises import pink_noise, white_noise, blue_noise

np.random.seed(42)

# Define the grid dimensions
nx = 128  # Number of points in x direction
ny = 128  # Number of points in y direction

# Define the range for x and y
x_min, x_max = 0, 1  # Range for x
y_min, y_max = 0, 1  # Range for y

# Create the vertices
x_vertices = np.linspace(x_min, x_max, nx+1)  # Create x values
y_vertices = np.linspace(y_min, y_max, ny+1)  # Create y values

# Use meshgrid to create the 2D vertices grid
XV, YV = np.meshgrid(x_vertices, y_vertices)

# Create the cell centers
x_centers = (x_vertices[:-1]+x_vertices[1:]) / 2.0  # Create x values
y_centers = (y_vertices[:-1]+y_vertices[1:]) / 2.0  # Create y values

# Use meshgrid to create the 2D grid
X, Y = np.meshgrid(x_centers, y_centers)

# Calculate the step sizes in x and y directions
dx = x_centers[1] - x_centers[0]
dy = y_centers[1] - y_centers[0]

# Calculate the wavenumbers in x and y directions
kx = 2 * np.pi * rfftfreq(nx, dx)
ky = 2 * np.pi * fftfreq(ny, dy)

# Use meshgrid to create the 2D grid
KX, KY = np.meshgrid(kx, ky)
K2 = KX**2 + KY**2
K = np.sqrt(K2)
INVK2 = np.where(K2==0, 0, 1.0/K2)
KNORM = np.sqrt((KX/np.max(KX))**2 + (KY/np.max(KY))**2)
SPEC_FILTER = np.exp(-23.6 * 1024 * (KNORM - 2.0/3.0)**4.)
SPEC_FILTER[KNORM <= 2.0/3.0] = 1

/var/folders/7s/tn4_jx8158vf7rpdxlfyncr40000gn/T/ipykernel_48148/2618192977.py:44: RuntimeWarning: divide by zero encountered in divide
  INVK2 = np.where(K2==0, 0, 1.0/K2)


In [58]:
def create_vtk_image_data(souts):
    # Create vtkImageData
    imageData = vtk.vtkImageData()
    imageData.SetDimensions(nx+1, ny+1, 1)
    imageData.SetSpacing(dx, dy, 1)
    imageData.SetOrigin(x_min, y_min, 0)
    
    for key in souts:
        # Create temperature data array
        tempArray = vtk.vtkDoubleArray()
        tempArray.SetName(key)
        tempArray.SetNumberOfComponents(1)
        tempArray.SetNumberOfTuples(nx * ny)
        tempArray.SetArray(souts[key].ravel(), nx * ny, 1)
        imageData.GetCellData().SetScalars(tempArray)
    
    return imageData

def write_vtk_image_data(imageData, filename):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(imageData)
    writer.SetDataMode(vtk.vtkXMLWriter.Binary)
    writer.Write()

In [59]:
OMEGAH = rfft2(pink_noise(ny, nx))
OMEGAH *= SPEC_FILTER
OMEGAH /= np.sqrt(0.5*np.sum(irfft2(1j*KY*OMEGAH * INVK2)**2+irfft2(1j*KX*OMEGAH * INVK2)**2)*dx*dy)
OMEGAH *= 0.1

PSIHAT = OMEGAH * INVK2
U = irfft2(1j*KY*PSIHAT)
V =-irfft2(1j*KX*PSIHAT)
E0 = 0.5*np.sum(U**2+V**2)*dx*dy

nu = 1e-5
nt = 101
tf = 10
t = 0
CFL = 0.5
dt = CFL/max(np.max(np.abs(U/dx)),np.max(np.abs(V/dy)))
nt_sub = int(np.ceil(tf/nt/dt))
print(dt, nt_sub, E0, np.max(U))

0.012522713876177361 8 0.010000000000000002 0.2617694445800932


In [60]:
def N(OMEGAH):
    PSIHAT = OMEGAH * INVK2
    NONL = -irfft2(1j*KY*PSIHAT)*irfft2(1j*KX*OMEGAH) + irfft2(1j*KX*PSIHAT)*irfft2(1j*KY*OMEGAH)
    NONL = rfft2(NONL) * SPEC_FILTER
    NONL += -nu * K2 * OMEGAH
    return NONL

for i in range(nt):
    for _ in range(nt_sub):
        k1 = dt * N(OMEGAH)
        k2 = dt * N(OMEGAH+1.0/2.0*k1)
        k3 = dt * N(OMEGAH+1.0/2.0*k2)
        k4 = dt * N(OMEGAH+k3)
        OMEGAH += 1.0/6.0*(k1 + 2*k2 + 2*k3 + k4)
        t += dt

    PSIHAT = OMEGAH * INVK2
    U = irfft2(1j*KY*PSIHAT)
    V =-irfft2(1j*KX*PSIHAT)
    OMEGA = irfft2(OMEGAH)
    E = 0.5*np.sum(U**2+V**2)*dx*dy
    CFL = dt*max(np.max(np.abs(U/dx)),np.max(np.abs(V/dy)))
    print(f"{i}, {t:.2f}, {CFL:.2f}, {E/E0:.4f}, {np.sum(OMEGA)}")

    imageData = create_vtk_image_data({"omega": OMEGA})
    filename = f"results/vort_transport_2d_{i:03d}.vti"
    write_vtk_image_data(imageData, filename)


0, 0.10, 0.47, 0.9981, 0.0
1, 0.20, 0.47, 0.9964, -4.547473508864641e-13
2, 0.30, 0.48, 0.9947, -4.547473508864641e-13
3, 0.40, 0.50, 0.9931, 0.0
4, 0.50, 0.51, 0.9917, -4.547473508864641e-13
5, 0.60, 0.50, 0.9903, 9.094947017729282e-13
6, 0.70, 0.50, 0.9890, 0.0
7, 0.80, 0.49, 0.9877, 0.0
8, 0.90, 0.47, 0.9865, -4.547473508864641e-13
9, 1.00, 0.46, 0.9854, -9.094947017729282e-13
10, 1.10, 0.43, 0.9843, -4.547473508864641e-13
11, 1.20, 0.43, 0.9833, 0.0
12, 1.30, 0.45, 0.9823, 0.0
13, 1.40, 0.47, 0.9813, -4.547473508864641e-13
14, 1.50, 0.48, 0.9804, -1.1368683772161603e-12
15, 1.60, 0.48, 0.9795, -1.1368683772161603e-12
16, 1.70, 0.48, 0.9787, -9.094947017729282e-13
17, 1.80, 0.48, 0.9779, -4.547473508864641e-13
18, 1.90, 0.47, 0.9771, -9.094947017729282e-13
19, 2.00, 0.46, 0.9764, -9.094947017729282e-13
20, 2.10, 0.46, 0.9756, -9.094947017729282e-13
21, 2.20, 0.47, 0.9749, -9.094947017729282e-13
22, 2.30, 0.48, 0.9742, 0.0
23, 2.40, 0.48, 0.9736, -9.094947017729282e-13
24, 2.50, 0.48